<a href="https://colab.research.google.com/github/pragya2003sah/ML-paper-replication/blob/main/paper_replication_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn

In [ ]:
class purple_block(nn.Module):
  def __init__(self,c1,c2):
    super().__init__()

    self.conv1=nn.Conv1d(in_channels=c1,out_channels=c2,kernel_size=2,stride=2)
    self.ln=nn.LayerNorm(c2)
    self.conv2=nn.Conv1d(in_channels=c2,out_channels=4*c2,kernel_size=1,stride=1)
    self.drop=nn.Dropout(p=0.2)

    self.conv=nn.Conv1d(in_channels=4*c2,out_channels=4*c2,kernel_size=1,stride=1)

    self.final_batch=nn.Sequential(
        nn.GELU(),
        nn.Conv1d(in_channels=8*c2,out_channels=c2,kernel_size=1,stride=1)
    )
  def forward(self,x):
    def first_batch(inp):
      z=self.conv1(inp)
      z=z.permute(0,2,1)
      z=self.ln(z)
      z=z.permute(0,2,1)#restoring the original shape
      z=self.conv2(z)
      z=self.drop(z)
      return z

    b1=first_batch(x)
    b2=first_batch(x)
    b3=first_batch(x)
    b1=self.conv(b1)
    b2=self.conv(b2)
    a=torch.cat((b1,b3),dim=1)
    b=torch.cat((b2,b3),dim=1)
    c=a+b
    c=self.final_batch(c)
    return c





In [ ]:
class Linear_Block(nn.Module):
  def __init__(self):
    super().__init__()
    self.L1=nn.Linear(in_features=512,out_features=256)
    self.G=nn.GELU()
    self.L2=nn.Linear(in_features=256,out_features=5)
  def forward(self,x):
    x=self.L1(x)
    x=self.G(x)
    x=self.L2(x)
    return x


In [ ]:
class Main_Architecture(nn.Module):
  def __init__(self):
    super().__init__()
    self.block1=purple_block(3,32)
    self.block2=purple_block(32,64)
    self.block3=purple_block(64,128)
    self.block4=purple_block(128,256)
    self.block5=purple_block(256,512)

    self.g=nn.AdaptiveAvgPool1d(1)

    self.block6=Linear_Block()

  def forward(self,x):
    y=self.block1(x)
    y=self.block2(y)
    y=self.block3(y)
    y=self.block4(y)
    y=self.block5(y)
    y=self.g(y)
    y=y.permute(0,2,1)
    y=self.block6(y)
    return y



In [ ]:
x=torch.randn(2,3,256)
model=Main_Architecture()
y=model(x)
y.shape

torch.Size([2, 1, 5])